In [1]:
import os
import sys

proj_root = os.path.abspath(os.getcwd())
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

In [2]:
# Zelle 2: HotelManager importieren und Instanz erstellen

from business_logic.hotel_manager import HotelManager
from data_access.hotel_data_access import HotelDataAccess
from data_access.room_data_access import RoomDataAccess
from data_access.booking_data_access import BookingDataAccess
from business_logic.room_manager import RoomManager
from data_access.facility_data_access import FacilityDataAccess

import model
from model import hotel

# Pfad zur SQLite-Datenbank:
db_path = os.path.join(proj_root, "database", "hotel_reservation_sample.db")

# DataAccess-Objekte erzeugen
hotel_dao = HotelDataAccess(db_path)
room_dao = RoomDataAccess(db_path)
booking_dao = BookingDataAccess(db_path)
facility_dao = FacilityDataAccess(db_path)

# HotelManager mit dem Datenbankpfad erzeugen
manager = HotelManager(hotel_data_access=hotel_dao, room_data_access=room_dao)
room_manager = RoomManager(room_data_access=room_dao, facility_data_access=facility_dao)

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:


1.1. Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [8]:
# DAO auf die DB zeigen lassen
dao = HotelDataAccess("database/hotel_reservation_sample.db")
manager = HotelManager(dao)

# nach Stadt filtern im String
hotels_in_WunschStadt = manager.show_hotels_by_city("Basel")

if not hotels_in_WunschStadt:
    print("Keine Hotels in dieser Stadt gefunden.")
else:
    for h in hotels_in_WunschStadt:
        print(f"{h.name} – {h.address.street}, {h.address.zip_code}, {h.address.city}")

Les Trois Rois – Freiestrasse 10, 4051, Basel


1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [9]:
dao = HotelDataAccess("database/hotel_reservation_sample.db")
manager = HotelManager(dao)

# Stadt und min_stars angeben
city = "Basel"
min_stars = 5

# 3) Methode aufrufen
result = manager.show_hotels_by_city_and_min_stars(city, min_stars)

# 4) Ausgabe, falls Treffer vorhanden
if result:
    print(f"Gefundene Hotels in '{city}' mit mindestens {min_stars} Sternen:")
    for h in result:
        print(f"  • {h.name} ({h.stars} Sterne) – {h.address.street}, {h.address.zip_code}")

Gefundene Hotels in 'Basel' mit mindestens 5 Sternen:
  • Les Trois Rois (5 Sterne) – Freiestrasse 10, 4051


1.3. Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung)

In [10]:
# 1) DAO-Objekte erstellen
dao = HotelDataAccess("database/hotel_reservation_sample.db")
room_dao = RoomDataAccess("database/hotel_reservation_sample.db")

# 2) HotelManager mit beiden DAOs initialisieren
manager = HotelManager(dao, room_dao)

# 3) Stadt und Gästeanzahl angeben
city = "zürich"
guest_count = 1

# 4) Methode aufrufen
result = manager.find_hotels_by_city_and_guests(city, guest_count)

# 5) Ausgabe, falls Treffer vorhanden
if result:
    print(f"Gefundene Hotels in '{city}' mit Zimmern für mindestens {guest_count} Gäste:")
    for h in result:
        print(f"  • {h.name} – {h.address.street}, {h.address.zip_code}")
else:
    print(f"Keine Hotels in '{city}' mit Zimmern für mindestens {guest_count} Gäste gefunden.")

Gefundene Hotels in 'zürich' mit Zimmern für mindestens 1 Gäste:
  • Hotel Baur au Lac – Bahnhofstrasse 1, 8001


1.4. Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.

In [11]:
from datetime import date

# Abfrageparameter definieren
city = "Genève"                 ## Beispiel Buchung in DB in Genève ist von 2025, 8, 20 bis  2025, 8, 22
check_in = date(2025, 8, 1)
check_out = date(2025, 8, 17)

# Verfügbare Hotels abrufen
verfuegbare_hotels = manager.find_available_hotels_by_city_and_dates(city, check_in, check_out, booking_dao)

# Ausgabe der Ergebnisse
if verfuegbare_hotels:
    print(f"Verfügbare Hotels in '{city}' vom {check_in} bis {check_out}:")
    for h in verfuegbare_hotels:
        print(f"{h.name} – {h.address.street}, {h.address.zip_code}")
else:
    print(f"Keine verfügbaren Hotels in '{city}' vom {check_in} bis {check_out}.")

Verfügbare Hotels in 'Genève' vom 2025-08-01 bis 2025-08-17:
Four Seasons Hôtel des Bergues – Rue du Rhône 42, 1204


1.5. Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne.


In [12]:
from datetime import datetime

# Parameter definieren --> Hotel in Genf hat 5 Sterne, Room für max 4 Leute uund booking von 2025-8-20 - 2025-8-22
stadt = "Genève"
check_in = datetime(2025, 8, 23)
check_out = datetime(2025, 8, 24)
min_sterne = 4
gaesteanzahl = 2

# Methode aufrufen
verfuegbare_hotels = manager.find_hotels_by_criteria(
    city=stadt,
    check_in_date=check_in,
    check_out_date=check_out,
    min_stars=min_sterne,
    guest_count=gaesteanzahl,
    booking_da=booking_dao
)

# Ausgabe
if verfuegbare_hotels:
    print(f"Verfügbare Hotels in '{stadt}' vom {check_in.date()} bis {check_out.date()} (mind. {min_sterne} Sterne, für {gaesteanzahl} Gäste):")
    for hotel in verfuegbare_hotels:
        print(f"  • {hotel.name} – {hotel.address.street}, {hotel.address.zip_code} {hotel.address.city}")
else:
    print(f"Keine passenden Hotels in '{stadt}' für die angegebenen Kriterien gefunden.")

Verfügbare Hotels in 'Genève' vom 2025-08-23 bis 2025-08-24 (mind. 4 Sterne, für 2 Gäste):
  • Four Seasons Hôtel des Bergues – Rue du Rhône 42, 1204 Genève


1.6. Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [13]:
infos = manager.show_all_hotel_infos()

print("Verfügbare Hotels:")
for eintrag in infos:
    print(" •", eintrag)


Verfügbare Hotels:
 • Hotel Baur au Lac – Bahnhofstrasse 1, 8001 Zürich (5 Sterne)
 • Four Seasons Hôtel des Bergues – Rue du Rhône 42, 1204 Genève (5 Sterne)
 • Grand Hotel National – Pilatusstrasse 15, 6003 Luzern (5 Sterne)
 • Bellevue Palace – Marktgasse 59, 3011 Bern (5 Sterne)
 • Les Trois Rois – Freiestrasse 10, 4051 Basel (5 Sterne)


2. Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.

2.1. Ich möchte die folgenden Informationen pro Zimmer
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung,
Ausstattung, Preis pro Nacht und Gesamtpreis.


In [6]:
from datetime import date

# Optional: Check-in und Check-out setzen (für Gesamtpreisberechnung)
check_in = date(2025, 7, 11)
check_out = date(2025, 7, 15)

# Zimmerdetails abrufen
room_infos = room_manager.show_room_details()

# Ausgabe
for room in room_infos:
    ausstattung = ", ".join([f.facility_name for f in room.facilities])

    print(f"Zimmer {room.room_number} – {room.room_type.description}")
    print(f"  Max. Gäste     : {room.room_type.max_guests}")
    print(f"  Ausstattung    : {ausstattung if ausstattung else 'Keine'}")

    # Preis pro Nacht
    print(f"  Preis pro Nacht: {room.price_per_night:.2f} CHF")

    # Optional: Gesamtpreis berechnen, wenn Check-in/Check-out gesetzt
    if check_in and check_out and check_out > check_in:
        nächte = (check_out - check_in).days
        gesamtpreis = room.price_per_night * nächte
        print(f"  Gesamtpreis    : {gesamtpreis:.2f} CHF für {nächte} Nächte")

    print("-" * 60)

Zimmer 101 – Single
  Max. Gäste     : 1
  Ausstattung    : WiFi, TV
  Preis pro Nacht: 250.00 CHF
  Gesamtpreis    : 1000.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 102 – Double
  Max. Gäste     : 2
  Ausstattung    : WiFi
  Preis pro Nacht: 400.00 CHF
  Gesamtpreis    : 1600.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 201 – Suite
  Max. Gäste     : 4
  Ausstattung    : Air Conditioning
  Preis pro Nacht: 650.00 CHF
  Gesamtpreis    : 2600.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 301 – Family Room
  Max. Gäste     : 5
  Ausstattung    : Mini Bar
  Preis pro Nacht: 900.00 CHF
  Gesamtpreis    : 3600.00 CHF für 4 Nächte
------------------------------------------------------------
Zimmer 401 – Penthouse
  Max. Gäste     : 6
  Ausstattung    : Keine
  Preis pro Nacht: 1500.00 CHF
  Gesamtpreis    : 6000.00 CHF für 4 Nächte
--------------------------------

2.2. Ich möchte nur die verfügbaren Zimmer sehen, sofern
ich meinen Aufenthalt (von – bis) spezifiziert habe.


In [5]:
from datetime import date

check_in = date(2025, 8, 20)
check_out = date(2025, 8, 22)

verfuegbare_zimmer = room_manager.find_available_rooms_by_dates(
    check_in_date=check_in,
    check_out_date=check_out,
    booking_dao=booking_dao
)

for room in verfuegbare_zimmer:
    ausstattung = ", ".join([f.facility_name for f in room.facilities])
    print(f"Zimmer {room.room_number} – {room.room_type.description}")
    print(f"  Hotel: {room.hotel.name}, Adresse: {room.hotel.address.street} {room.hotel.address.city}")
    print(f"  Max. Gäste     : {room.room_type.max_guests}")
    print(f"  Ausstattung    : {ausstattung if ausstattung else 'Keine'}")
    print(f"  Preis pro Nacht: {room.price_per_night:.2f} CHF")

    # Optional: Gesamtpreis berechnen
    if check_in and check_out and check_out > check_in:
        nächte = (check_out - check_in).days
        gesamtpreis = room.price_per_night * nächte
        print(f"  Gesamtpreis    : {gesamtpreis:.2f} CHF für {nächte} Nächte")

    print("-" * 60)

Zimmer 101 – Single
  Hotel: Hotel Baur au Lac, Adresse: Bahnhofstrasse 1 Zürich
  Max. Gäste     : 1
  Ausstattung    : WiFi, TV
  Preis pro Nacht: 250.00 CHF
  Gesamtpreis    : 500.00 CHF für 2 Nächte
------------------------------------------------------------
Zimmer 102 – Double
  Hotel: Hotel Baur au Lac, Adresse: Bahnhofstrasse 1 Zürich
  Max. Gäste     : 2
  Ausstattung    : WiFi
  Preis pro Nacht: 400.00 CHF
  Gesamtpreis    : 800.00 CHF für 2 Nächte
------------------------------------------------------------
Zimmer 301 – Family Room
  Hotel: Grand Hotel National, Adresse: Pilatusstrasse 15 Luzern
  Max. Gäste     : 5
  Ausstattung    : Mini Bar
  Preis pro Nacht: 900.00 CHF
  Gesamtpreis    : 1800.00 CHF für 2 Nächte
------------------------------------------------------------
Zimmer 401 – Penthouse
  Hotel: Bellevue Palace, Adresse: Marktgasse 59 Bern
  Max. Gäste     : 6
  Ausstattung    : Keine
  Preis pro Nacht: 1500.00 CHF
  Gesamtpreis    : 3000.00 CHF für 2 Nächte
----